In [1]:
import matplotlib as mpl
mpl.use("pgf")
# from palettable.tableau import Tableau_10
# from cycler import cycler
import numpy as np
pgf_with_pdflatex = {
    "pgf.texsystem": "pdflatex",
    "pgf.preamble": [
         r"\usepackage[utf8x]{inputenc}",
         r"\usepackage[T1]{fontenc}",
         r"\usepackage{mathptmx}",
         r"\usepackage{mathtools}",
#          r"\usepackage{biblatex}",
         ],
    "text.usetex": True,
    "figure.figsize": [4, 3.75],
    "axes.labelsize": 12,
    "font.size": 12,
    "font.family": "serif",
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "lines.linewidth": 1.5,
#     "axes.prop_cycle": cycler(color=Tableau_10.mpl_colors),
    "xtick.major.size": 5,
    "xtick.major.width": 1,
    "xtick.minor.size": 2.5,
    "xtick.minor.width": 1,
    "ytick.major.size": 5,
    "ytick.major.width": 1,
    "ytick.minor.size": 2.5,
    "ytick.minor.width": 1,
}
mpl.rcParams.update(pgf_with_pdflatex)

from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, AutoMinorLocator

In [3]:
def antoine(T, A, B, C):
    return A - B/(T - C)
ortega_data = np.loadtxt('vapor_pressure/ortega.csv', delimiter=',')
vanGenderen_data = np.loadtxt('vapor_pressure/vanGenderen.csv', delimiter=',')
verevkin_data = np.loadtxt('vapor_pressure/verevkin.csv', delimiter=',')
all_data = np.vstack((ortega_data, vanGenderen_data, verevkin_data))
all_data = all_data[all_data[:, 0].argsort()]
coeffs, cov_mat = curve_fit(antoine, all_data[:, 0], np.log10(all_data[:, 1]), p0=[6, 1500, 50], maxfev=10000)
print(coeffs)
conf_int = 2*np.sqrt(np.diag(cov_mat))
print(conf_int)
ortega_fit, _ = curve_fit(antoine, ortega_data[:, 0], np.log10(ortega_data[:, 1]), p0=[6, 1500, 50])
print(ortega_fit)

[    6.40297589  1528.68913524    52.88138973]
[  0.09186087  53.47122789   4.93379075]
[    6.21051367  1415.38995238    63.91337448]


In [4]:
ortega = np.array([6.23175, 1429.00, 62.30])
camacho = np.array([5.9644, 1281.06, 75.94])
stephenson = np.array([6.62646, 1658.4, 42.09])

In [5]:
temps = np.arange(np.floor(all_data[0, 0]), np.ceil(all_data[-1, 0]), 1)

In [6]:
fig, ax = plt.subplots()
ax.set_yscale('log')
ax.set_xlabel('$T$, K', labelpad=5.0)
ax.set_ylabel(r'$P_{\text{sat}}$, kPa', labelpad=2.0)
ax.set_ylim(1E-1, 2E2)
ax.set_xlim(270, 420)
ax.xaxis.set_major_locator(MaxNLocator(6))
ax.xaxis.set_minor_locator(AutoMinorLocator(3))
# ax.plot(temps, 10**antoine(temps, *ortega), label=r"Ortega et al.\ \cite{Ortega2003}")
ax.plot(ortega_data[:, 0], ortega_data[:, 1], marker='o', linestyle='None', label=r"Ortega et al.\ \cite{Ortega2003}", markersize=3)
# ax.plot(temps, 10**antoine(temps, *camacho), label=r"Camacho et al.\ \cite{Camacho2007}")
# ax.plot(temps, 10**antoine(temps, *stephenson), label=r"Stephenson et al.\ \cite{Stephenson1987}")
ax.plot(vanGenderen_data[:, 0], vanGenderen_data[:, 1], marker='x', linestyle='None', label=r"van Genderen et al.\ \cite{vanGenderen2002}")
ax.plot(verevkin_data[:, 0], verevkin_data[:, 1], marker='+', linestyle='None', label=r"Verevkin and Emel'yanenko \cite{Verevkin2008}")
ax.plot(temps, 10**antoine(temps, *coeffs), label="This Work")
ax.legend(loc='best');

In [7]:
fig.savefig('vapor_pressure.pgf', bbox_inches='tight')